# Children Act applications csv

### This code summarises the information produced from the children act applications code into tidy data sets, and combines them all at the end.
1. Individual children by order type
2. Orders applied for
3. Applications (events)
4. Case starts (including applicants/respondents and high court indicator)
5. Individual children (annual)
6. Individual children (quarterly)

In [ ]:
import os  # for file paths
import pandas as pd
import awswrangler as wr
import pydbtools as pydb  # see https://github.com/moj-analytical-services/pydbtools
import altair as alt  # for plotting, see https://altair-viz.github.io/getting_started/overview.html

# few things for viewing dataframes better
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [ ]:
# define some variables to be used throughout the notebook
db2 = "fcsq"

In [ ]:
#Applications by type - children involved
pydb.create_temp_table( 
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) Quarter,
  'Application' AS Type,
  'Children' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
   Order_desc AS Order_type,
   Order_code AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  {db2}.ca_apps_child
Where
  Year > 2010
Group BY
  Year,
  Quarter,
  Case_type,
  Order_desc,
  Order_code
""",

"ca_apps_type_child")

In [ ]:
#Orders applied for (by order type)
pydb.create_temp_table(
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) Quarter,
  'Application' AS Type,
  'Order type' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
   Order_desc,
   Order_code AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  {db2}.ca_apps_order_types
Where
  Year > 2010
Group BY
  Year,
  Quarter,
  Case_type,
  Order_desc,
  Order_code
""",

"ca_apps_type_order")

In [ ]:
#Application events
pydb.create_temp_table(
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) Quarter,
  'Application' AS Type,
  'Application events' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
   'All' AS Order_type,
   CAST(NULL AS INTEGER) AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_apps_event_count
Where
  Year > 2010
Group BY
  Year,
  Quarter,
  Case_type
""",

"ca_apps_events")

In [ ]:
#Case starts - including number of applicants/respondents and Hight court indicator*/
pydb.create_temp_table(
f"""
SELECT
  P.Year,
  CAST(P.Quarter AS VARCHAR) Quarter,
  'Cases' AS Type,
  'Cases starting' AS Count_type,
  CASE WHEN P.Case_type = 'C' THEN 'Public law'
       WHEN P.Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
  'All' AS Order_type,
   CAST(NULL AS INTEGER) AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
   Case When P.no_of_applicants = 1 THEN '1'
      ELSE '2+'
    END AS Applicants_in_case,
   Case When P.no_of_respondents in (1,2,3)
         THEN CAST(P.no_of_respondents AS VARCHAR)
         ELSE '4+'
        END AS Respondents_in_case,
   CASE WHEN HC.case_hc_value IS NULL THEN 'No'
          ELSE HC.HC_London_Ind
     END AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.party_count P
  left join __temp__.ca_cases_HC HC
    on P.Case_number = HC.Case_number
     and P.case_type = HC.case_type
Where
  P.Year > 2010
Group BY
  P.Year,
  P.Quarter,
  p.case_type,
  Case When P.no_of_applicants = 1 THEN '1'
      ELSE '2+'
    END,
  Case When P.no_of_respondents in (1,2,3)
         THEN CAST(P.no_of_respondents AS VARCHAR)
         ELSE '4+'
        END,      
  CASE WHEN HC.case_hc_value IS NULL THEN 'No'
          ELSE HC.HC_London_Ind
     END
""",

"ca_apps_cases")

In [ ]:
#Individual children - annual - including age of child
pydb.create_temp_table(
f"""
SELECT
  Year,
  CAST(NULL AS VARCHAR) AS Quarter,
  'Application' AS Type,
  'Individual children' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
  'All' AS Order_type,
  CAST(NULL AS INTEGER) AS Order_type_code,
  Gender,
  age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
  COUNT(*) AS Count
FROM
  __temp__.ca_apps_ind_child_yr
Where
  Year > 2010
Group BY
  Year,
  Case_type,
  Gender,
  age_band
""",

"ca_apps_ind_yr")

In [ ]:
#Individual children - quarterly - including age of child
pydb.create_temp_table(
f"""
SELECT
  CAST(NULL AS INTEGER) AS Year,
  CONCAT(CAST(year AS VARCHAR),'-',CAST(quarter AS VARCHAR)) AS Quarter,
  'Application' AS Type,
  'Individual children' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
  'All' AS Order_type,
  CAST(NULL AS INTEGER) AS Order_type_code,
  Gender,
  age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
  COUNT(*) AS Count
FROM
  __temp__.ca_apps_ind_child_qtr
Where
  Year > 2010
Group BY
  Year,
  Quarter,
  Case_type,
  Gender,
  age_band
""",

"ca_apps_ind_qtr")

In [ ]:
#Union together datasets
pydb.create_temp_table(
f"""
SELECT
  *
FROM
  __temp__.CA_APPS_TYPE_CHILD
UNION ALL
SELECT
  *
FROM
  __temp__.CA_APPS_TYPE_ORDER
UNION ALL
SELECT
  *
FROM
  __temp__.CA_APPS_EVENTS
UNION ALL
SELECT
  *
FROM
  __temp__.CA_APPS_CASES
UNION ALL
SELECT
  *
FROM
  __temp__.CA_APPS_IND_YR
UNION ALL
SELECT
  *
FROM
  __temp__.CA_APPS_IND_QTR

""",
    
"ca_apps_csv")

### Load to dataframe and put in S3

In [ ]:
data = pydb.read_sql_query ("select * from __temp__.ca_apps_csv")

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv ('Exports/apps_data_with_no_child.csv', header = True)

In [ ]:
df.to_csv (r's3://alpha-family-data/fcsq_processing/CA_apps/test_csv/ca_apps.csv', header = True)